In [330]:
import numpy as np
import pandas as pd
import pickle
import os 
import cx_Oracle
from datetime import datetime

# Read Data Files

In [331]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN")

In [332]:
pos_df = pd.read_csv(os.getcwd()+r"\Data Files\pos lat lng.csv",encoding = 'latin')

In [333]:
cus_df = pd.read_pickle(os.getcwd()+r"\Data Files\final_grids_combined.pkl")

In [334]:
cus_pos_info = pd.read_csv(os.getcwd()+r"\Data Files\skp_case_pos_mapping.csv",encoding = 'latin')

In [335]:
cus_df = cus_df.merge(cus_pos_info, on = 'SKP_CREDIT_CASE', how = 'inner')
cus_df = cus_df.merge(pd.DataFrame(pos_df[["SKP_SALESR","LAT","LONG_"]]).rename(columns = 
            {"SKP_SALESR":"SKP_SALESROOM","LAT":"CUS_POS_LAT","LONG_":"CUS_POS_LONG"}), on = 'SKP_SALESROOM', how = 'inner')


In [336]:
cus_df.dropna(subset = ["CUS_LAT","CUS_LONG","CUS_POS_LAT","CUS_POS_LONG"], inplace = True)

In [337]:
pos_df.set_index('SKP_SALESR', inplace = True)

In [338]:
cus_df = cus_df[cus_df.TIME_DECISION_DATE>=datetime(2019,3,1)]

# Define Functions

In [340]:
from math import cos, sqrt
def qick_distance(Lat1,Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1) * cos((Lat2 + Lat1)*0.00872664626)  
    return 111.319 * sqrt(x*x + y*y)

In [341]:
# from math import radians, cos, sin, asin, sqrt
# def haversine(lat1, lon1, lat2, lon2):
#     """
#     Calculate the great circle distance between two points 
#     on the earth (specified in decimal degrees)
#     """
#     # convert decimal degrees to radians 
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

#     # haversine formula 
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * asin(sqrt(a)) 
#     r = 6371 # Radius of earth in kilometers. Use 3956 for miles
#     return c * r

In [342]:
from numpy import radians, cos, sin, arcsin, sqrt

def haversine(lat1,lon1,lat2,lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """

    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    dlat = np.subtract(lat2, lat1)

    a = np.add(np.power(np.sin(np.divide(dlat, 2)), 2),  
                          np.multiply(np.cos(lat1), 
                                      np.multiply(np.cos(lat2), 
                                                  np.power(np.sin(np.divide(dlon, 2)), 2))))
    c = np.multiply(2, np.arcsin(np.sqrt(a)))
    r = 6371

    return c*r

In [343]:
# def euclidean_distance(row1, row2):
#         distance = 0.0
#         for i in range(len(row1)):
#             distance += (row1[i] - row2[i])**2
#         return sqrt(distance)
    
# def get_neighbors(train, test_row, num_neighbors):
#         distances = list()
#         for train_row in train.index:
#             dist = euclidean_distance(test_row, [train["LAT"][train_row],train["LONG_"][train_row]])
#             distances.append((train_row, dist))
#         distances.sort(key=lambda tup: tup[1])
#         neighbors = list()
#         for i in range(num_neighbors):
#             neighbors.append(distances[i])
#         return neighbors


# def get_rank_order(train, test_row, SKP_SALESR):
#         distances = list()
#         for train_row in train.index:
#             dist = euclidean_distance(test_row, [train["LAT"][train_row],train["LONG_"][train_row]])    
#             distances.append((train_row, dist))
#         distances.sort(key=lambda tup: tup[1])

#         for i in range(train.shape[0]):
#             if SKP_SALESR == distances[i][0]:
#                 return i+1

#         return np.NaN


# def get_nearest_pos(train, test_row):
#         distances = list()
#         for train_row in train.index:
#             dist = euclidean_distance(test_row, [train["LAT"][train_row],train["LONG_"][train_row]])
#             distances.append((train_row, dist))
#         distances.sort(key=lambda tup: tup[1])

#         return distances[0][0]

    



### Use vecor operations in functions instead of scalers, to speed up 100X 

In [344]:
def get_rank_order(train, test_row, SKP_SALESR):
        distances = np.sqrt((test_row[0] - train.iloc[:, 0]) ** 2 + (test_row[1] - train.iloc[:, 1]) ** 2)
        distances.sort_values(inplace = True)
        
#         distances.sort(key=lambda tup: tup[1])
        t = 1
        for i in distances.index:
            if SKP_SALESR == i:
                return t
            t+=1

        return np.nan

    
def get_nearest_pos(train, test_row):
        distances = np.sqrt((test_row[0] - train.iloc[:, 0]) ** 2 + (test_row[1] - train.iloc[:, 1]) ** 2)
        distances.sort_values(inplace = True)

        return distances.index[0]


    
def get_neighbors(train, test_row, num_neighbors):
        distances = np.sqrt((test_row[0] - train.iloc[:, 0]) ** 2 + (test_row[1] - train.iloc[:, 1]) ** 2)
        distances.sort_values(inplace = True)
        
        neighbors = list()
        for i in range(num_neighbors):
            neighbors.append(distances.index[i])
        return neighbors


### Create combined function 

In [345]:
def get_combined(train, test_row, SKP_SALESR):
        distances = np.sqrt((test_row[0] - train.iloc[:, 0]) ** 2 + (test_row[1] - train.iloc[:, 1]) ** 2)
        distances.sort_values(inplace = True)
        
#         distances.sort(key=lambda tup: tup[1])
        pos_order = np.nan
        t = 1
        for i in distances.index:
            if SKP_SALESR == i:
                pos_order = t
                break
            t+=1

        return {'POS_ORDER':pos_order,"NEAREST_POS":distances.index[0],
                        "500m":len(distances[distances<0.005]),
                        "1km":len(distances[distances<0.01]),
                        "2km":len(distances[distances<0.02]),
                        "3km":len(distances[distances<0.03]),
                        "5km":len(distances[distances<0.05])
                       }


# Generate Features

### Rank order of POS

In [346]:
tmp = {"SKP_CREDIT_CASE":[],"POS_ORDER":[], "NEAREST_POS":[],"POS_500m":[],
      "POS_1km":[],
      "POS_2km":[],
      "POS_3km":[],
      "POS_5km":[]}
start_time = datetime.now()
for x in cus_df.index:
    tmp["SKP_CREDIT_CASE"].append(cus_df.SKP_CREDIT_CASE[x])
    t = get_combined(pos_df[["LAT","LONG_"]],[cus_df.CUS_LAT[x],cus_df.CUS_LONG[x]],cus_df.SKP_SALESROOM[x])
    tmp["POS_ORDER"].append(t["POS_ORDER"])
    tmp["NEAREST_POS"].append(t["NEAREST_POS"])
    tmp["POS_500m"].append(t["500m"])
    tmp["POS_1km"].append(t["1km"])
    tmp["POS_2km"].append(t["2km"])
    tmp["POS_3km"].append(t["3km"])
    tmp["POS_5km"].append(t["5km"])
    
print("time taken in s " ,  (datetime.now()-start_time).seconds)

time taken in s  12718


In [347]:
cus_df = cus_df.merge(pd.DataFrame(tmp), on = 'SKP_CREDIT_CASE', how= 'inner')


In [348]:
cus_df.head()

,SKP_CREDIT_CASE,TIME_DECISION_DATE,CUS_LAT,CUS_LONG,RISK_FPD10,RISK_FPD30,RISK_FSPD30,RISK_FSTPD30,RISK_FSTQPD30,RISK_FSTQPD60,...,SKP_SALESROOM,CUS_POS_LAT,CUS_POS_LONG,POS_ORDER,NEAREST_POS,POS_500m,POS_1km,POS_2km,POS_3km,POS_5km
0,245663258,2019-08-04,30.817513,75.188828,0.0,0.0,0.0,0.0,0.0,0.0,...,263979,30.817887,75.172352,35,4789380,2,9,54,74,75
1,227547736,2019-03-05,30.901959,75.233321,0.0,0.0,0.0,0.0,0.0,0.0,...,263979,30.817887,75.172352,55,7909763,0,0,0,0,11
2,227701333,2019-03-06,30.807835,75.182451,0.0,0.0,0.0,0.0,0.0,0.0,...,263979,30.817887,75.172352,23,38089080,0,7,69,75,75
3,228284193,2019-03-11,30.819649,75.104914,0.0,0.0,0.0,0.0,0.0,0.0,...,263979,30.817887,75.172352,36,34465125,0,0,0,0,0
4,228715739,2019-03-14,30.804140,75.176426,0.0,0.0,0.0,0.0,0.0,0.0,...,263979,30.817887,75.172352,32,24595728,0,8,67,74,75


In [349]:
cus_df.to_csv(os.getcwd()+r"\Data Files\backup1.csv")

### Distance from POS

In [350]:
cus_df["DISTANCE_FROM_POS"] = haversine(cus_df.CUS_LAT,cus_df.CUS_LONG,cus_df.CUS_POS_LAT,cus_df.CUS_POS_LONG)

### Distance from nearest POS

In [351]:
pos_df1 = pd.read_csv(os.getcwd()+r"\Data Files\pos lat lng.csv",encoding = 'latin')

In [352]:
cus_df = cus_df.merge(pd.DataFrame(pos_df1[["SKP_SALESR","LAT","LONG_"]]).rename(columns = 
            {"SKP_SALESR":"NEAREST_POS","LAT":"NEAREST_POS_LAT","LONG_":"NEAREST_POS_LONG"}), on = 'NEAREST_POS', how = 'inner')


In [353]:
cus_df["DISTANCE_FROM_NEAREST_POS"] = haversine(cus_df.CUS_LAT,cus_df.CUS_LONG,cus_df.NEAREST_POS_LAT,cus_df.NEAREST_POS_LONG)

In [354]:
cus_df.to_csv(os.getcwd()+r"\Data Files\backup2.csv")

### People in vicinity - 500m, 1KM, 2KM, 3KM, 5KM radius

In [355]:
#Factor of 100 between haversine and eucledian distance

In [356]:
cus_train=cus_df.sample(50000,random_state=2)

In [366]:
tmp = {"SKP_CREDIT_CASE":[],"CUST_500m":[],
      "CUST_1km":[],
      "CUST_2km":[],
      "CUST_3km":[],
      "CUST_5km":[]}
start_time = datetime.now()
for x in cus_df.index:
    tmp["SKP_CREDIT_CASE"].append(cus_df.SKP_CREDIT_CASE[x])
    t = get_combined(cus_train[["CUS_LAT","CUS_LONG"]],[cus_df.CUS_LAT[x],cus_df.CUS_LONG[x]],cus_df.SKP_SALESROOM[x])
    tmp["CUST_500m"].append(t["500m"])
    tmp["CUST_1km"].append(t["1km"])
    tmp["CUST_2km"].append(t["2km"])
    tmp["CUST_3km"].append(t["3km"])
    tmp["CUST_5km"].append(t["5km"])
    
    if x%100000==0:
        tmp1=pd.DataFrame(tmp)
        tmp1.to_csv(os.getcwd()+r"\Data Files\backup3.csv")
        print("time taken in s " ,  (datetime.now()-start_time).seconds)
        
print("time taken in s " ,  (datetime.now()-start_time).seconds)

time taken in s  0
time taken in s  6775
time taken in s  12704
time taken in s  15463
time taken in s  18105
time taken in s  20698
time taken in s  23252
time taken in s  25779
time taken in s  28309
time taken in s  30863
time taken in s  33450
time taken in s  39799
time taken in s  44934
time taken in s  48839
time taken in s  52249


In [ ]:
cus_df = cus_df.merge(pd.DataFrame(tmp), on = 'SKP_CREDIT_CASE', how= 'inner')

In [ ]:
cus_df.to_csv(os.getcwd()+r"\Data Files\backup3.csv")

### POS in vicinity - 500m, 1KM, 2KM, 3KM, 5KM radius

In [287]:
#Features already generated in joint function call while POS_ORDERING generation

### POC for distance between PA and CA